In [1]:
import sys
import logging
import numpy as np
import matplotlib.pyplot as plt
import jax.numpy as jnp
#sys.path.insert(0, "insert path")

from rubix.galaxy.input_handler.nihao import NihaoHandler
from rubix.core.data import convert_to_rubix, prepare_input
from rubix.core.telescope import get_spatial_bin_edges, get_spaxel_assignment
from rubix.utils import SFTtoAge


In [ ]:
handler = NihaoHandler(
    #path='insert path to snapshot data',
    #halo_path='insert path to halo data',
)
data = handler.get_data()

In [3]:
# Beispiel-Konfiguration für Rubix
config = {
    "logger": {
        "log_level": "DEBUG",
        "log_file_path": None,
        "format": "%(asctime)s - %(name)s - %(levelname)s - %(message)s",
    },
    "data": {
        "name": "NihaoHandler",
        "args": {
            #"path": "insert path to snapshot data",
            #"halo_path": "insert path to halo data",
        },
    },
    "simulation": {
        "name": "NIHAO",
        "args": {
            #"path": "insert path to snapshot data",
            #"halo_path": "insert path to halo data",
        },
    },
    "output_path": "output",
    "telescope": {
        "name": "MUSE",
    },
    "cosmology": {
        "name": "PLANCK15",
    },
    "galaxy": {
        "dist_z": 0.1,
    }
}

In [ ]:
from rubix.core.data import convert_to_rubix, prepare_input
convert_to_rubix(config) # Convert the config to rubix format and store in output_path folder
coords, vel, metallicity, mass, age, additional_value = prepare_input(config)

In [ ]:
n_subset = 10000
subset = coords[:n_subset]
plt.scatter(subset[:,0], subset[:,1], s=0.5)
plt.xlabel('X Position (kpc)')
plt.ylabel('Y Position (kpc)')
plt.title('Scatter Plot of Star Positions in NIHAO Galaxy')
plt.show()

In [ ]:
#create a density plot of the star positions
spatial_bin_edges = get_spatial_bin_edges(config)
bin_particles = get_spaxel_assignment(config)

In [ ]:
# filter coords
mask = (
    (coords[:, 0] >= spatial_bin_edges[0]) &
    (coords[:, 0] <= spatial_bin_edges[-1]) &
    (coords[:, 1] >= spatial_bin_edges[0]) &
    (coords[:, 1] <= spatial_bin_edges[-1])
)
filtered_coords = coords[mask]

# assign spaxels
spaxel_result = bin_particles({"coords": filtered_coords})
spaxel_index = spaxel_result["pixel_assignment"]


In [ ]:
# create density image
image = np.zeros((len(spatial_bin_edges) - 1, len(spatial_bin_edges) - 1))
for i in range(len(spatial_bin_edges) - 1):
    for j in range(len(spatial_bin_edges) - 1):
        image[i, j] = np.sum(spaxel_index == (i + (len(spatial_bin_edges) - 1) * j))

# plot density image
image = np.log10(image + 1)
plt.imshow(image, cmap="viridis")
plt.colorbar(label="Particle Density")
plt.xlabel("X Spaxel")
plt.ylabel("Y Spaxel")
plt.title("Binned Density Image of NIHAO Galaxy Stars")
plt.show()